In [0]:
%run /Workspace/Users/simon.nagy@hiflylabs.com/.bundle/dbxSandbox/dev/files/sandboxProject/includeFiles/Classroom-Setup

In [0]:
source_json_path = f"{datasets_dir}/events/events-500k.json"
df = (
    spark
    .read
    .option("inferSchema", True)
    .json(source_json_path)
)

output_dir = working_dir + "/delta/events"
(df
 .write
 .format("delta")
 .mode("overwrite")
 .save(output_dir))

In [0]:
display(df)

In [0]:
df.schema

## Column expressions

In [0]:
from pyspark.sql.functions import col

print(col("device"))

creating expressions with columns, using the standard operators

In [0]:
col("ecommerce.purchase_revenue_in_usd") + col("ecommerce.total_item_quantity")

In [0]:
clean_df = (
    df
    .filter(col("ecommerce.purchase_revenue_in_usd").isNotNull())
    .withColumn("purchase_revenue", (col("ecommerce.purchase_revenue_in_usd")*100).cast("int"))
    .sort(col("purchase_revenue").desc())
)

display(clean_df)

## DataFrame Transformation methods
- select, drop, groupBy etc.

In [0]:
from pyspark.sql.functions import col

location_df = clean_df.select(
  "user_id" # col is not needed, when column is selected without manipulation
  ,col("geo.city").alias("city")
  ,col("geo.state").alias("state")
)

display(location_df)

In [0]:
anonymous_df = df.drop("user_id", "geo", "device")
display(anonymous_df)

## Adding / replacing columns

- `withColumn()` returns a new dataframe by adding a column or replacing an existing column, that has the same name
- `withColumnRenamed()` returns a new dataframe with renamed columns
- `filter()`
- `dropDuplicates()` returns a dataframe with removed duplicates, optionally considering a subset of columns; technically alias for distinct; but can be contorlled better, looking at unique values in column
- `limit()` returns a dataframe with only first n rows
- `sort()` returns a dataframe sorted <= alias for orderBy, which is sorting all memory partitions>; sort() is only sorting on a given memory partition
-- major sort / minor sort; 2 cols can be added to orderBy, and it combines them during sorting

In [0]:
mobile_df = df.withColumn("mobile", col("device").isin("iOS", "Android"))
# mobile col is appended on the end
display(mobile_df)

In [0]:
mobile_df = mobile_df.filter(col("mobile") == True)
display(mobile_df)

In [0]:
classroom_cleanup()